In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score

from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW

from transformers import get_scheduler
from datasets import Dataset

import os

# Load dataset
df = pd.read_csv("/kaggle/input/news-dataset/News_Category_Dataset.csv")
df = df[['headline', 'category']].dropna()

# Encode category labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['category'])

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df[['headline', 'label']])

# Split
dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset['train']
test_dataset = dataset['test']

# Tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch['headline'], padding='max_length', truncation=True, max_length=64)

# Tokenize
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Rename for HuggingFace compatibility
train_dataset = train_dataset.rename_column("label", "labels")
test_dataset = test_dataset.rename_column("label", "labels")

# Format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# Load BERT model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_encoder.classes_))

# Optimizer & Scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
num_training_steps = len(train_loader) * 3
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training
print("Training model...")
model.train()
for epoch in range(4):
    total_loss = 0
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

# Evaluation
print("Evaluating model...")
model.eval()
y_preds, y_true = [], []
with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        y_preds.extend(predictions.cpu().numpy())
        y_true.extend(batch['labels'].cpu().numpy())

# Metrics
acc = accuracy_score(y_true, y_preds)
report = classification_report(y_true, y_preds, target_names=label_encoder.classes_)

# Save classification report
os.makedirs("bert_outputs", exist_ok=True)
with open("bert_outputs/classification_report.txt", "w") as f:
    f.write(f"Accuracy: {acc:.4f}\n\n")
    f.write(report)

print("Accuracy:", acc)
print("\nClassification Report saved to: bert_outputs/classification_report.txt")

# Save fine-tuned model and tokenizer
model.save_pretrained("bert_outputs/bert_headline_model")
tokenizer.save_pretrained("bert_outputs/bert_headline_model")
print("Model and tokenizer saved to: bert_outputs/bert_headline_model/")


2025-05-09 18:29:59.320873: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746815399.618388      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746815399.689594      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Map:   0%|          | 0/41905 [00:00<?, ? examples/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training model...
Epoch 1, Loss: 15627.5079
Epoch 2, Loss: 10977.5438
Epoch 3, Loss: 8575.6009
Epoch 4, Loss: 7633.0947
Evaluating model...
Accuracy: 0.6584894404009068

Classification Report saved to: bert_outputs/classification_report.txt
Model and tokenizer saved to: bert_outputs/bert_headline_model/
